# See [Vehicle Routing Problem with Time Windows](https://developers.google.com/optimization/routing/vrptw#time_window_constraints)




[時間枠付き巡回セールスマン問題](https://scmopt.github.io/opt100/58twtsp.html)

```
mtz: model for the traveling salesman problem with time windows
(based on Miller-Tucker-Zemlin's one-index potential formulation, stronger constraints)
Parameters:
    - n: number of nodes
    - c[i,j]: cost for traversing arc (i,j)
    - e[i]: earliest date for visiting node i
    - l[i]: latest date for visiting node i
Returns a model, ready to be solved.


    model = Model("tsptw - mtz-strong")
    x, u = {}, {}
    for i in range(1, n + 1):
        u[i] = model.addVar(lb=e[i], ub=l[i], vtype="C", name="u(%s)" % i)
        for j in range(1, n + 1):
            if i != j:
                x[i, j] = model.addVar(vtype="B", name="x(%s,%s)" % (i, j))
    model.update()

    for i in range(1, n + 1):
        model.addConstr(
            quicksum(x[i, j] for j in range(1, n + 1) if j != i) == 1, "Out(%s)" % i
        )
        model.addConstr(
            quicksum(x[j, i] for j in range(1, n + 1) if j != i) == 1, "In(%s)" % i
        )

        for j in range(2, n + 1):
            if i != j:
                M1 = max(l[i] + c[i, j] - e[j], 0)
                M2 = max(l[i] + min(-c[j, i], e[j] - e[i]) - e[j], 0)
                model.addConstr(
                    u[i] + c[i, j] - M1 * (1 - x[i, j]) + M2 * x[j, i] <= u[j],
                    "LiftedMTZ(%s,%s)" % (i, j),
                )

    for i in range(2, n + 1):
        model.addConstr(
            e[i]
            + quicksum(
                max(e[j] + c[j, i] - e[i], 0) * x[j, i]
                for j in range(1, n + 1)
                if i != j
            )
            <= u[i],
            "LiftedLB(%s)" % i,
        )

        model.addConstr(
            u[i]
            <= l[i]
            - quicksum(
                max(l[i] - l[j] + c[i, j], 0) * x[i, j]
                for j in range(2, n + 1)
                if i != j
            ),
            "LiftedUB(%s)" % i,
        )

    model.setObjective(quicksum(c[i, j] * x[i, j] for (i, j) in x), GRB.MINIMIZE)

    model.update()
    model.__data = x, u
    return model
```



In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

I = 4
T = 100
t_list = [15, 20, 50, 5]
t = dict((i, t_list[i-1]) for i in range(1, I+1))
a_list = [10, 1, 24, 4]
a = dict((i, a_list[i-1]) for i in range(1, I+1))


M = ConcreteModel("Days allocation")
M.I = Set(initialize=range(1, I+1))
M.x = Var(M.I, within=NonNegativeIntegers)
M.x_sum = sum(M.x[i] for i in M.I)
def const_rule1(model):
    return M.x_sum <= T
def const_rule2(model, i):
    return M.x[i] >= t[i]
M.const1 = Constraint(rule=const_rule1)
M.const2 = Constraint(M.I, rule=const_rule2)
def obj_rule(model):
    return sum(a[i] * M.x[i] + 1.0 for i in M.I)
M.value = Objective(rule=obj_rule, sense=maximize)

opt = SolverFactory("glpk")
result = opt.solve(M, tee=True)
M.display()

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp1jxmu05i.glpk.raw --wglp /tmp/tmpltnfyzi8.glpk.glp --cpxlp
 /tmp/tmpxbhu9wi2.pyomo.lp
Reading problem data from '/tmp/tmpxbhu9wi2.pyomo.lp'...
6 rows, 5 columns, 9 non-zeros
4 integer variables, none of which are binary
49 lines were read
Writing problem data to '/tmp/tmpltnfyzi8.glpk.glp'...
39 lines were written
GLPK Integer Optimizer 5.0
6 rows, 5 columns, 9 non-zeros
4 integer variables, none of which are binary
Preprocessing...
1 row, 4 columns, 4 non-zeros
4 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
1 row, 4 columns, 4 non-zeros
*     0: obj =   1.394000000e+03 inf =   0.000e+00 (4)
*     2: obj =   1.634000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
In